This notebook is for calculating distances, elevations and timings between Dovetrek checkpoints. It then produces a routecard for a set route.

###Contents:
* [Imports](#Imports)
* [Start SparkSession](#Start-SparkSession)
* [Set Parameters](#set-params)
* [Get Secrets](#get-secrets)
* [Fetch Checkpoint Information](#fetch-cp-info)
* [Convert BNG to NGR](#bgn-to-ngr)
* [Convert NGR to Latitude/Logitude](#ngr-to-latlong)
* [Create Matrix of Checkpoint combinations](#cp-combos)
* [Fetch Distances & Elevations from Google Maps API](#D&E-google)
* [Fetch Distances & Elevations from OpenRouteService API](#D&E-ors)
* [Fetch Distances & Elevations from Bing Maps API](#D&E-bing)
* [Save Distances_DF as csv in memory](#)
* [Upload Distances_DF csv to GitHub filestore](#)

### Imports

In [70]:
from pyspark.sql.functions import col, expr, first
from pyspark.sql import functions as F, DataFrame, Window, SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType, ArrayType, TimestampType
from datetime import datetime, time, timedelta
import requests
import time
import ipywidgets
import base64
import getpass
import builtins
import json
from io import BytesIO, StringIO
import itertools
from functools import reduce
import numpy as np

### Start SparkSession

In [71]:
try:
    SparkSession.active()
except:
    spark = SparkSession.builder \
        .appName("BinderPySpark") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .getOrCreate()
else:
    SparkSession.active()
finally:
    # Test if Spark is working
    df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
    df.show()
    df.unpersist()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



### Get available competition years

In [72]:
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/CheckpointData"
params = {"ref": "main"}
response = requests.get(url, params=params)
response.json()
# Extract the 'name' field from each item in the JSON response
Year_List = [item['name'] for item in response.json() if 'name' in item]

# Extract the year using string slicing
Year_List = [filename.split('_')[1].split('.')[0] for filename in Year_List]

### Set Parameters

In [100]:
API_Service_List = ["Bing Maps","Google Maps", "OpenRouteService","Azure Maps & OpenTopoData"]

Competition_Year_Picker = ipywidgets.Dropdown(options=Year_List, value = '2025', description = 'Pick a competition year:')

API_Service_Picker = ipywidgets.Dropdown(options=API_Service_List, value = "Google Maps", description = 'Pick an API Service for distance & elevations:')

display(Competition_Year_Picker)
display(API_Service_Picker)
print("Make sure all of the parameters are set correctly before entering your GitHub Secrets repository Personal Access Token below")
GitHubPAT = getpass.getpass("Token:")


Dropdown(description='Pick a competition year:', index=4, options=('2017', '2018', '2019', '2024', '2025'), va…

Dropdown(description='Pick an API Service for distance & elevations:', index=1, options=('Bing Maps', 'Google …

Make sure all of the parameters are set correctly before entering your GitHub Secrets repository Personal Access Token below
Token:··········


### Check parameters correctly set

In [101]:
Competition_Year = Competition_Year_Picker.value
API_Service = API_Service_Picker.value
print(f"Competition Year: {Competition_Year} \nAPI Service: {API_Service}")

Competition Year: 2017 
API Service: Bing Maps


### Get Secrets

In [59]:
# 🔹 GitHub API URL to get the file
url = f"https://api.github.com/repos/liamj-f/Secrets/contents/Dovetrek_Secrets.json"

# 🔹 GitHub API headers
headers = {"Authorization": f"token {GitHubPAT}"}

# 🔹 Fetch the secrets file
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Decode Base64 content
    content = response.json()["content"]
    decoded_content = base64.b64decode(content).decode("utf-8")

    # Load JSON into a dictionary
    secrets = json.loads(decoded_content)

    # 🔹 Assign values to variables
    BingMapsAPIKey = secrets.get("BingMApsAPIKey", "")
    AzureMapsAPIKey = secrets.get("AzureMapsAPIKey", "")
    OrdnanceSurveyAPIKey = secrets.get("OrdnanceSurveyAPIKey", "")
    OpenRouteServiceAPIKey = secrets.get("OpenRouteServiceAPIKey", "")
    GoogleMapsAPIKey = secrets.get("GoogleMapsAPIKey", "")
    DovetrekRepoPAT = secrets.get("DovetrekRepoPAT", "")

    print("✅ Secrets loaded successfully!")

else:
    print(f"❌ Error: {response.status_code} - {response.text}")


✅ Secrets loaded successfully!


### Fetch Checkpoint information

In [60]:
# GitHub API URL for file content
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/CheckpointData/Openings_{Competition_Year}.csv"

# Fetch file content
response = requests.get(url, headers=headers)
content = response.json()["content"]
decoded_content = base64.b64decode(content).decode("utf-8")
csv_lines = decoded_content.split("\n")
# Create an RDD from the list
rdd = spark.sparkContext.parallelize(csv_lines)
# Convert RDD to DataFrame
openings_df = spark.read.csv(rdd, header=True, inferSchema=True)

### Convert BNG to NGR
The actual grid square identifiers are missing from the datasets because Dovetrek always takes place in the same area, so conversion from letter to number not necessary.

In [61]:
openings_df = openings_df.withColumn("NGR_Easting", F.concat(F.lit("4"), F.substring(openings_df.BNG, 1, 3), F.lit("00")).cast("double")) \
                    .withColumn("NGR_Northing", F.concat(F.lit("3"), F.substring(openings_df.BNG, 5, 3), F.lit("00")).cast("double"))
display(openings_df)

DataFrame[CP: string, BNG: string, 1000: int, 1030: int, 1100: int, 1130: int, 1200: int, 1230: int, 1300: int, 1330: int, 1400: int, 1430: int, 1500: int, 1530: int, 1600: int, 1630: int, 1700
: double, NGR_Easting: double, NGR_Northing: double]

### Convert NGR to Latitude/Longitude

In [62]:
LatLong_Df = spark.createDataFrame(
    [(1, 1.0, 1.0)],
    schema='CP string, Latitude float, Longitude float'
)
LatLong_Df = LatLong_Df.filter('1!=1')

# Convert DataFrame to a list
gridrefs_list = openings_df.select(
    openings_df.CP,
    openings_df.NGR_Easting,
    openings_df.NGR_Northing
).collect()

# Iterate over the list
for row in gridrefs_list:
    # Perform desired operations with the column values
    resp = requests.get(
        "http://webapps.bgs.ac.uk/data/webservices/CoordConvert_LL_BNG.cfc?method=BNGtoLatLng&easting="
        + str(row.NGR_Easting)
        + "&northing="
        + str(row.NGR_Northing)
    )

    # Parse the JSON response
    json_resp = json.loads(resp.text)

    # Check if 'LATITUDE' and 'LONGITUDE' keys exist in the response
    if 'LATITUDE' in json_resp and 'LONGITUDE' in json_resp:
        # Extract latitude and longitude values
        latitude = json_resp['LATITUDE']
        longitude = json_resp['LONGITUDE']
        cp = row["CP"]

        df = spark.createDataFrame(
            [(cp, latitude, longitude)],
            'CP string, Latitude float, Longitude float'
        )
        LatLong_Df = LatLong_Df.union(df)

openings_df = openings_df.join(LatLong_Df, openings_df.CP == LatLong_Df.CP).drop(LatLong_Df.CP)

LatLong_Df.unpersist()
# Show DataFrame
display(openings_df)

DataFrame[CP: string, BNG: string, 1000: int, 1030: int, 1100: int, 1130: int, 1200: int, 1230: int, 1300: int, 1330: int, 1400: int, 1430: int, 1500: int, 1530: int, 1600: int, 1630: int, 1700
: double, NGR_Easting: double, NGR_Northing: double, Latitude: float, Longitude: float]

### Create Matrix of Checkpoint combinations

In [63]:
# So this isn't the best way to do this, i realise, but it is reused code from another part of the project and doesn't take too long so we may as well just get every combination and run it through the api

StartCPs = openings_df.select("CP").withColumnRenamed("CP","StartCP")
FinishCPs = openings_df.select("CP").withColumnRenamed("CP","FinishCP")
CP_Combinations_DF = StartCPs.crossJoin(FinishCPs)

StartCPs.unpersist
FinishCPs.unpersist

CP_Combinations_DF = CP_Combinations_DF.alias("df").join(openings_df.select("CP", "Latitude", "Longitude").withColumnRenamed('Latitude','StartLatitude').withColumnRenamed('Longitude','StartLongitude'), CP_Combinations_DF.StartCP == openings_df.CP).drop("CP")

CP_Combinations_DF = CP_Combinations_DF.alias("df").join(openings_df.select("CP", "Latitude", "Longitude").withColumnRenamed('Latitude','FinishLatitude').withColumnRenamed('Longitude','FinishLongitude'), CP_Combinations_DF.FinishCP == openings_df.CP).drop("CP")

CP_Combinations_DF = CP_Combinations_DF.filter(CP_Combinations_DF.StartCP != CP_Combinations_DF.FinishCP)
display(CP_Combinations_DF)

DataFrame[StartCP: string, FinishCP: string, StartLatitude: float, StartLongitude: float, FinishLatitude: float, FinishLongitude: float]

### Fetch Distances & Elevations from Google Maps API

In [64]:
if API_Service == "Google Maps":

  CP_Combinations_List = CP_Combinations_DF.collect()

  Distances_DF = spark.createDataFrame(
      [(1, 1.0, 1.0, 1)],
      schema='StartCP string, FinishCP string, Distance float, Height_Gain int'
  )
  Distances_DF = Distances_DF.filter("1!=1")

  def make_request(method, url, **kwargs):
      """Helper function to handle API requests with 429 retry logic."""
      while True:
          response = requests.request(method, url, **kwargs)
          if response.status_code == 429:
              print("Rate limit exceeded (429). Waiting 60 seconds before retrying...")
              time.sleep(60)
          else:
              return response  # Return response if successful

  for row in CP_Combinations_List:
      wp1 = f"{row.StartLongitude},{row.StartLatitude}"
      wp2 = f"{row.FinishLongitude},{row.FinishLatitude}"

      # Route API Call with Retry Handling
      route_url = "https://routes.googleapis.com/directions/v2:computeRoutes"
      route_params = {
    "origin":{
      "location":{
        "latLng":{
          "latitude": row.StartLatitude,
          "longitude": row.StartLongitude
        }
      }
    },
    "destination":{
      "location":{
        "latLng":{
          "latitude": row.FinishLatitude,
          "longitude": row.FinishLongitude
        }
      }
    },
    "travelMode": "WALK",
    "languageCode": "en-GB",
    "units": "METRIC"
  }
      route_headers = {
          "X-Goog-Api-Key": GoogleMapsAPIKey,
          "X-Goog-FieldMask": "routes.duration,routes.distanceMeters,routes.polyline,routes.legs"
      }

      route_resp = make_request("POST", route_url, headers=route_headers, json=route_params)
      route_json_resp = route_resp.json()

      travel_distance = float(route_json_resp["routes"][0]["legs"][0]["distanceMeters"]) / 1000
      StartCP, FinishCP = row["StartCP"], row["FinishCP"]

      # Prepare height points for elevation request
      height_polyline = route_json_resp["routes"][0]["polyline"]["encodedPolyline"]


      # Elevation API Call with Retry Handling
      elevations_url = "https://maps.googleapis.com/maps/api/elevation/json"
      elevations_params = {
          "path": "enc:"+height_polyline,
          "samples": 100,
          "key": GoogleMapsAPIKey
      }

      elevations_resp = make_request("GET", elevations_url, params=elevations_params)
      elevation_json_resp = elevations_resp.json()
      elevations = [point["elevation"] for point in elevation_json_resp["results"]]
      differences = [builtins.max(0, elevations[i+1] - elevations[i]) for i in range(len(elevations)-1)]
      height_gain = float(sum(differences))

      # Append to Spark DataFrame
      df = spark.createDataFrame(
          [(StartCP, FinishCP, travel_distance, height_gain)],
          'StartCP string, FinishCP string, Distance float, Height_Gain float'
      )
      Distances_DF = Distances_DF.union(df)

  #   time.sleep(2.0)  # Keep a delay between requests to reduce rate limits

  display(Distances_DF)

else:
    print(f"Skipping Google Maps Cell, selected service: {API_Service}")

DataFrame[StartCP: string, FinishCP: string, Distance: float, Height_Gain: float]

### Fetch Distances & Elevations from Azure Maps and OpenTopoData APIs

In [65]:
# Takes ~17 minutes due to OpenTopoData API limits 1 per second
if API_Service == "Azure Maps & OpenTopoData":

    CP_Combinations_List = CP_Combinations_DF.collect()

    Distances_DF = spark.createDataFrame(
        [(1, 1.0, 1.0, 1)],
        schema='StartCP string, FinishCP string, Distance float, Height_Gain int'
    )
    Distances_DF = Distances_DF.filter("1!=1")

    MAX_LOCATIONS = 100  # OpenTopoData API limit

    for row in CP_Combinations_List:
        wp1 = f"{row.StartLatitude},{row.StartLongitude}"
        wp2 = f"{row.FinishLatitude},{row.FinishLongitude}"

        # Azure Maps Routing API
        route_url = "https://atlas.microsoft.com/route/directions/json"
        route_params = {
            "subscription-key": AzureMapsAPIKey,
            "api-version": "1.0",
            "query": f"{wp1}:{wp2}",
            "travelMode": "pedestrian",
            "routeType": "shortest",
            "traffic": "false",
            "computeBestOrder": "false",
            "computeTravelTimeFor": "all",
        }
        route_resp = requests.get(route_url, params=route_params)
        route_json_resp = json.loads(route_resp.text)

        travel_distance = float(route_json_resp['routes'][0]['summary']['lengthInMeters']) / 1000  # Convert meters to km
        StartCP = row["StartCP"]
        FinishCP = row["FinishCP"]

        # Extract route coordinates
        height_points = route_json_resp['routes'][0]['legs'][0]['points']
        coordinates = [f"{pt['latitude']},{pt['longitude']}" for pt in height_points]

        # Batch coordinates into chunks of 100
        elevation_results = []
        for i in range(0, len(coordinates), MAX_LOCATIONS):
            chunk = "|".join(coordinates[i:i + MAX_LOCATIONS])  # Format chunk

            # OpenTopoData API call
            opentopo_url = "https://api.opentopodata.org/v1/eudem25m"  # Example dataset
            opentopo_params = {"locations": chunk}
            elevations_resp = requests.get(opentopo_url, params=opentopo_params)
            elevations_json_resp = json.loads(elevations_resp.text)

            # Extract elevations
            if 'results' in elevations_json_resp:
                elevation_results.extend([result['elevation'] for result in elevations_json_resp['results']])

            time.sleep(1)

        # Calculate height gain
        differences = [max(0, elevation_results[i+1] - elevation_results[i]) for i in range(len(elevation_results)-1)]
        height_gain = float(sum(differences))

        df = spark.createDataFrame(
            [(StartCP, FinishCP, travel_distance, height_gain)],
            'StartCP string, FinishCP string, Distance float, Height_Gain float'
        )
        Distances_DF = Distances_DF.union(df)

    display(Distances_DF)

else:
    print(f"Skipping Azure Maps Cell, selected service: {API_Service}")


Skipping Azure Maps Cell, selected service: Google Maps


### Fetch Distances & Elevations from OpenRouteService API

In [66]:
if API_Service == "OpenRouteService":

    CP_Combinations_List = CP_Combinations_DF.collect()

    Distances_DF = spark.createDataFrame(
        [(1, 1.0, 1.0, 1)],
        schema='StartCP string, FinishCP string, Distance float, Height_Gain int'
    )
    Distances_DF = Distances_DF.filter("1!=1")

    def make_request(method, url, **kwargs):
        """Helper function to handle API requests with 429 retry logic."""
        while True:
            response = requests.request(method, url, **kwargs)
            if response.status_code == 429:
                print("Rate limit exceeded (429). Waiting 60 seconds before retrying...")
                time.sleep(60)
            else:
                return response  # Return response if successful

    for row in CP_Combinations_List:
        wp1 = f"{row.StartLongitude},{row.StartLatitude}"
        wp2 = f"{row.FinishLongitude},{row.FinishLatitude}"

        # Route API Call with Retry Handling
        route_url = "https://api.openrouteservice.org/v2/directions/foot-hiking"
        route_params = {
            "start": wp1,
            "end": wp2,
            "api_key": OpenRouteServiceAPIKey
        }
        route_resp = make_request("GET", route_url, params=route_params)
        route_json_resp = route_resp.json()

        travel_distance = float(route_json_resp["features"][0]["properties"]["summary"]["distance"]) / 1000
        StartCP, FinishCP = row["StartCP"], row["FinishCP"]

        # Prepare height points for elevation request
        height_points = route_json_resp["features"][0]["geometry"]["coordinates"]
        flattened_height_points = ",".join([f"{lon},{lat}" for lon, lat in height_points])

        # Elevation API Call with Retry Handling
        elevations_url = "https://api.openrouteservice.org/elevation/line"
        elevations_params = {
            "format_in": "polyline",
            "format_out": "polyline",
            "dataset": "srtm",
            "geometry": height_points
        }
        elevations_headers = {
            "Authorization": OpenRouteServiceAPIKey,
            "Content-Type": "application/json"
        }
        elevations_resp = make_request("POST", elevations_url, headers=elevations_headers, json=elevations_params)
        elevation_json_resp = elevations_resp.json()
        elevation_coordinates = elevation_json_resp["geometry"]
        elevations = [elev for _, _, elev in elevation_coordinates]
        differences = [builtins.max(0, elevations[i+1] - elevations[i]) for i in range(len(elevations)-1)]
        height_gain = sum(differences)

        # Append to Spark DataFrame
        df = spark.createDataFrame(
            [(StartCP, FinishCP, travel_distance, height_gain)],
            'StartCP string, FinishCP string, Distance float, Height_Gain float'
        )
        Distances_DF = Distances_DF.union(df)

        time.sleep(2.0)  # Keep a delay between requests to reduce rate limits

    display(Distances_DF)

else:
    print(f"Skipping OpenRouteService Cell, selected service: {API_Service}")

Skipping OpenRouteService Cell, selected service: Google Maps


### Fetch Distances & Elevations from Bing Maps API

In [67]:
if API_Service == "Bing Maps":

    CP_Combinations_List = CP_Combinations_DF.collect()

    Distances_DF = spark.createDataFrame(
        [(1, 1.0, 1.0, 1)],
        schema='StartCP string, FinishCP string, Distance float, Height_Gain int'
    )
    Distances_DF = Distances_DF.filter("1!=1")

    for row in CP_Combinations_List:
        wp1 = str(row.StartLatitude) + ',' + str(row.StartLongitude)
        wp2 = str(row.FinishLatitude) + ',' + str(row.FinishLongitude)

        route_url = "http://dev.virtualearth.net/REST/v1/Routes/walking"
        route_params = {
            "wayPoint.1": wp1,
            "waypoint.2": wp2,
            "optimize": "distance",
            "avoid": "ferry",
            "routeAttributes": "routePath,excludeItinerary",
            "distanceUnit": "km",
            "key": BingMapsAPIKey
        }
        route_resp = requests.get(route_url, params=route_params)
        route_json_resp = json.loads(route_resp.text)
        travel_distance = float(route_json_resp['resourceSets'][0]['resources'][0]['travelDistance'])
        StartCP = row["StartCP"]
        FinishCP = row["FinishCP"]

        height_points = route_json_resp['resourceSets'][0]['resources'][0]['routePath']['line']['coordinates']
        flattened_height_points = ",".join([f"{lat},{lon}" for lat, lon in height_points])  # Fix formatting

        elevations_url = "http://dev.virtualearth.net/REST/v1/Elevation/List"
        elevations_params = {
            "points": flattened_height_points,
            "heights": "ellipsoid",
            "key": BingMapsAPIKey
        }
        elevations_resp = requests.get(elevations_url, params=elevations_params)
        elevations_json_resp = json.loads(elevations_resp.text)
        elevations = elevations_json_resp['resourceSets'][0]['resources'][0]['elevations']
        differences = [builtins.max(0, elevations[i+1] - elevations[i]) for i in range(len(elevations)-1)]
        height_gain = sum(differences)

        df = spark.createDataFrame(
            [(StartCP, FinishCP, travel_distance, height_gain)],
            'StartCP string, FinishCP string, Distance float, Height_Gain int'
        )
        Distances_DF = Distances_DF.union(df)

    display(Distances_DF)

else:
    print(f"Skipping BingMaps Cell, selected service: {API_Service}")

Skipping BingMaps Cell, selected service: Google Maps


### Fetch Distances & Elevations from Azure Maps & OpenTopoData

In [68]:
# Takes ~17 minutes due to OpenTopoData API limits 1 per second
if API_Service == "Azure Maps & OpenTopoData":

    CP_Combinations_List = CP_Combinations_DF.collect()

    Distances_DF = spark.createDataFrame(
        [(1, 1.0, 1.0, 1)],
        schema='StartCP string, FinishCP string, Distance float, Height_Gain int'
    )
    Distances_DF = Distances_DF.filter("1!=1")

    MAX_LOCATIONS = 100  # OpenTopoData API limit

    for row in CP_Combinations_List:
        wp1 = f"{row.StartLatitude},{row.StartLongitude}"
        wp2 = f"{row.FinishLatitude},{row.FinishLongitude}"

        # Azure Maps Routing API
        route_url = "https://atlas.microsoft.com/route/directions/json"
        route_params = {
            "subscription-key": AzureMapsAPIKey,
            "api-version": "1.0",
            "query": f"{wp1}:{wp2}",
            "travelMode": "pedestrian",
            "routeType": "shortest",
            "traffic": "false",
            "computeBestOrder": "false",
            "computeTravelTimeFor": "all",
        }
        route_resp = requests.get(route_url, params=route_params)
        route_json_resp = json.loads(route_resp.text)

        travel_distance = float(route_json_resp['routes'][0]['summary']['lengthInMeters']) / 1000  # Convert meters to km
        StartCP = row["StartCP"]
        FinishCP = row["FinishCP"]

        # Extract route coordinates
        height_points = route_json_resp['routes'][0]['legs'][0]['points']
        coordinates = [f"{pt['latitude']},{pt['longitude']}" for pt in height_points]

        # Batch coordinates into chunks of 100
        elevation_results = []
        for i in range(0, len(coordinates), MAX_LOCATIONS):
            chunk = "|".join(coordinates[i:i + MAX_LOCATIONS])  # Format chunk

            # OpenTopoData API call
            opentopo_url = "https://api.opentopodata.org/v1/eudem25m"  # Example dataset
            opentopo_params = {"locations": chunk}
            elevations_resp = requests.get(opentopo_url, params=opentopo_params)
            elevations_json_resp = json.loads(elevations_resp.text)

            # Extract elevations
            if 'results' in elevations_json_resp:
                elevation_results.extend([result['elevation'] for result in elevations_json_resp['results']])

            time.sleep(1)

        # Calculate height gain
        differences = [max(0, elevation_results[i+1] - elevation_results[i]) for i in range(len(elevation_results)-1)]
        height_gain = float(sum(differences))

        df = spark.createDataFrame(
            [(StartCP, FinishCP, travel_distance, height_gain)],
            'StartCP string, FinishCP string, Distance float, Height_Gain float'
        )
        Distances_DF = Distances_DF.union(df)

    display(Distances_DF)

else:
    print(f"Skipping Azure Maps Cell, selected service: {API_Service}")


Skipping Azure Maps Cell, selected service: Google Maps


### Save Distances_DF as csv in memory

In [69]:
# 🔹 Save image to an in-memory buffer
buf = BytesIO()
Distances_DF.toPandas().to_csv(buf, index=False)
buf.seek(0)  # Move to start of buffer

# 🔹 Convert buffer to Base64
encoded_content = base64.b64encode(buf.read()).decode("utf-8")

filename = f"Distances_DF_{Competition_Year}_{API_Service}_{datetime.today().date()}.csv"

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/conversion.py", line 280, in _collect_as_arrow
    results = list(batch_stream)
              ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/serializers.py", line 69, in load_stream
    for batch in self.serializer.load_stream(stream):
  File "/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/serializers.py", line 112, in load_stream
    for batch in reader:
  File "pyarrow/ipc.pxi", line 671, in pyarrow.lib.RecordBatchReader.__next__
  File "pyarrow/ipc.pxi", line 705, in pyarrow.lib.RecordBatchReader.read_next_batch
  File "pyarrow/error.pxi", line 89, in pyarrow.lib.check_status
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception 

KeyboardInterrupt: 

### Upload Distances_DF csv to GitHub filestore

In [ ]:
# 🔹 GitHub API URL
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/{filename}"

# Get the latest SHA (if the file exists)
response = requests.get(url, headers={"Authorization": f"token {DovetrekRepoPAT}"})
sha = response.json().get("sha") if response.status_code == 200 else None

# 🔹 Upload file to GitHub
payload = {
    "message": f"Auto-upload {filename} from notebook run",
    "content": encoded_content,
    "branch": "FileStore"
}
if sha:
    payload["sha"] = sha  # Required if updating an existing file

response = requests.put(url, headers={"Authorization": f"token {DovetrekRepoPAT}"}, json=payload)

if response.status_code in [200, 201]:
    print(f"✅ {filename} uploaded successfully to Dovetrek/DataFrames (Filestore branch)!")
else:
    print(f"❌ Error: {response.status_code} - {response.text}")